<a href="https://colab.research.google.com/github/frag30041997/jupyter-one/blob/Cours-Data-Analyst/TP_final_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import seaborn as sns; sns.set
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder as LE
import numpy as np
import os


def accuracy(preds,target):
  return (preds==target).sum()/len(target)


def show_cm(confusion_matrix, labels):
    df_cm = pd.DataFrame(confusion_matrix, labels, labels)
    sns.heatmap(df_cm, annot=True)
    plt.show()

#fonction qui reécrit une colonne pour la refaire en int 
def sci_to_int(feature):
  iplist = []
  for i in range(len(feature)):
    iplist.append(int(feature[i]))
  return iplist

df_r6s5 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/datasets/r6_s5.csv',encoding="ISO-8859-1",engine='python')

print(df_r6s5.shape)

(4999, 31)


#Preprocessing

Le dataset est trop volumineux mais on ne peut pas retirer des données au hasard car les données sont groupés par "matchid" et on veux des donnés sur plusieurs dates, il va falloir commencer par un gros travail de preprocessing qui définira grandement le rendu par effet papillon.

#Phase 2 (transfo data)

In [16]:
df_r6s5.head(15)

,dateid,platform,gamemode,mapname,matchid,roundnumber,objectivelocation,winrole,endroundreason,roundduration,clearancelevel,skillrank,role,team,haswon,operator,nbkills,isdead,primaryweapon,primaryweapontype,primarysight,primarygrip,primaryunderbarrel,primarybarrel,secondaryweapon,secondaryweapontype,secondarysight,secondarygrip,secondaryunderbarrel,secondarybarrel,secondarygadget
0,2017-02-12,PC,HOSTAGE,CLUB_HOUSE,1522380841,1,STRIP_CLUB,Defender,AttackersKilledHostage,124,64,Gold,Defender,1,1,SWAT-CASTLE,0,0,UMP45,Submachine_Guns,RedDot,Vertical,None,Compensator,5.7_USG,Pistols,None,None,None,None,IMPACT_GRENADE
1,2017-02-12,PC,HOSTAGE,CLUB_HOUSE,1522380841,4,CHURCH,Defender,AttackersEliminated,217,81,Gold,Defender,0,1,GSG9-JAGER,0,1,416-C_CARBINE,Assault_Rifles,RedDot,Vertical,Laser,Suppressor,P12,Pistols,None,None,Laser,Suppressor,DEPLOYABLE_SHIELD
2,2017-02-12,PC,HOSTAGE,CLUB_HOUSE,1522380841,3,CHURCH,Defender,AttackersEliminated,160,150,Gold,Defender,1,1,JTF2-FROST,0,0,9mm_C1,Submachine_Guns,Reflex,None,None,None,MK1_9mm,Pistols,None,None,None,None,DEPLOYABLE_SHIELD
3,2017-02-12,PC,HOSTAGE,CLUB_HOUSE,1522380841,4,CHURCH,Defender,AttackersEliminated,217,94,Gold,Defender,0,1,BOPE-CAVEIRA,3,0,M12,Submachine_Guns,RedDot,None,None,MuzzleBrake,PRB92,Pistols,None,None,None,None,IMPACT_GRENADE
4,2017-02-12,PC,HOSTAGE,CLUB_HOUSE,1522380841,6,BEDROOM,Attacker,DefendersEliminated,143,81,Gold,Defender,0,0,GSG9-JAGER,0,1,416-C_CARBINE,Assault_Rifles,RedDot,Vertical,Laser,Suppressor,P12,Pistols,None,None,Laser,Suppressor,DEPLOYABLE_SHIELD
5,2017-02-12,PC,HOSTAGE,CLUB_HOUSE,1522380841,2,CHURCH,Attacker,DefendersEliminated,129,81,Gold,Defender,0,0,GSG9-JAGER,0,1,416-C_CARBINE,Assault_Rifles,RedDot,Vertical,Laser,Suppressor,P12,Pistols,None,None,Laser,Suppressor,DEPLOYABLE_SHIELD
6,2017-02-12,PC,HOSTAGE,CLUB_HOUSE,1522380841,5,CASH_ROOM,Attacker,DefendersEliminated,175,120,Gold,Attacker,0,1,G.E.O.-JACKAL,2,0,C7E,Assault_Rifles,Acog,Vertical,None,MuzzleBrake,ITA12S,Shotguns,Reflex,None,Laser,None,BREACH_CHARGE
7,2017-02-12,PC,HOSTAGE,CLUB_HOUSE,1522380841,3,CHURCH,Defender,AttackersEliminated,160,83,Gold,Defender,1,1,GSG9-JAGER,1,0,416-C_CARBINE,Assault_Rifles,Acog,Vertical,None,MuzzleBrake,P12,Pistols,None,None,None,None,DEPLOYABLE_SHIELD
8,2017-02-12,PC,HOSTAGE,CLUB_HOUSE,1522380841,1,STRIP_CLUB,Defender,AttackersKilledHostage,124,120,Gold,Attacker,0,0,GIGN-TWITCH,0,1,F2,Assault_Rifles,Acog,Vertical,None,Compensator,P9,Pistols,None,None,None,MuzzleBrake,BREACH_CHARGE
9,2017-02-12,PC,HOSTAGE,CLUB_HOUSE,1522380841,6,BEDROOM,Attacker,DefendersEliminated,143,64,Gold,Attacker,1,1,SWAT-ASH,1,0,R4-C,Assault_Rifles,Acog,Vertical,None,Compensator,5.7_USG,Pistols,None,None,None,None,BREACH_CHARGE
